In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/project_folderv2

/content/drive/MyDrive/project_folderv2


In [ ]:
!pip install Flask==3.0.0
!pip install pyngrok==7.1.2
!pip install opencv-python torch numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 2.5 MB/s eta 0:00:00
  Attempting uninstall: blinker
    Found existing installation: blinker 1.4
ERROR: Cannot uninstall 'blinker'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-

In [14]:
#gui
import os
from flask import Flask, render_template, request, send_from_directory
import subprocess
import base64
import time

app = Flask(__name__)

UPLOAD_PATH = '/content/drive/MyDrive/project_folderv2/static/upload'
if not os.path.exists(UPLOAD_PATH):
    os.makedirs(UPLOAD_PATH)

names = ['helmet', 'no-helmet', 'no-vest', 'person', 'vest']

@app.route('/', methods=['GET', 'POST'])
def application():
    if request.method == 'POST':
        upload_file = request.files['image_name']
        filename = upload_file.filename
        filepath = os.path.join(UPLOAD_PATH, filename)

        # Save the uploaded file
        upload_file.save(filepath)

        script_path = '/content/drive/MyDrive/project_folderv2/detect.py'
        args = ['python', script_path, '--source', filepath, '--weights', 'best.pt']
        process = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        output, error = process.communicate()
        process.wait()

        # Parse the output to extract the list of safety equipment detected
        safety_equipment_detected = []
        for line in output.decode('utf-8').splitlines():
            words = line.split()
            if any(word.isdigit() for word in words):  # Check if the line contains any digits
                detection_results = [f"{word} {words[idx + 1]}" for idx, word in enumerate(words) if word.isdigit()]
                safety_equipment_detected = ' Detected: ' + ' '.join(detection_results)
                break

        # Alarm logic: Check if a person is detected without a helmet or vest
        alarm_triggered = False
        if 'person' in safety_equipment_detected:
            if 'no-helmet' in safety_equipment_detected or 'no-vest' in safety_equipment_detected:
                alarm_triggered = True

        # Find the most recently created directory in /content/drive/MyDrive/project_folderv2/runs/detect/
        output_dir = max([f.path for f in os.scandir('/content/drive/MyDrive/project_folderv2/runs/detect/') if f.is_dir()], key=os.path.getmtime)

        # Read the output image file
        output_image_path = os.path.join(output_dir, filename)
        with open(output_image_path, 'rb') as f:
            image_data = f.read()
        image_b64 = base64.b64encode(image_data).decode('utf-8')

        timestamp = int(time.time())
        image_url = f"data:image/jpeg;base64,{image_b64}#{timestamp}"

        return render_template('result.html', safety_equipment_detected=safety_equipment_detected, image_url=image_url, alarm_triggered=alarm_triggered)

    return render_template('index.html')

@app.route('/uploaded_file/<filename>')
def uploaded_file(filename):
    return send_from_directory('./runs/detect', filename)

if __name__ == "__main__":
    # Setup ngrok for public URL
    from pyngrok import ngrok
    ngrok.set_auth_token("auth_token")
    http_tunnel = ngrok.connect(5000)
    print('Public URL:', http_tunnel.public_url)

    app.run(port=5000)

Public URL: https://dc9b-34-23-5-107.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [20/Jul/2024 16:53:59] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Jul/2024 16:54:00] "GET /static/css/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Jul/2024 16:54:00] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [20/Jul/2024 16:54:57] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [20/Jul/2024 16:54:57] "GET /static/css/style.css HTTP/1.1" 304 -
